In [1]:
import pandas as pd
data = pd.read_csv(r"C:\Users\mally\Leiden University [2022 -2024]\3rd Sem\API\Final Project\songSectionDataClean.csv")

In [2]:
def preprocess(data):
    output = []
    another_output = []
    for i in data:
        lst = i.split("-")
        if(len(lst)==1):
            continue
        else:
            output.append(' '.join(lst))
            another_output.append(lst[0])
    return output,another_output

In [3]:
newdata = pd.DataFrame()
newdata["Progression list"],output = preprocess(data["Progression"])

In [4]:
len(data["Name"].unique())

59

In [5]:
newdata["Progression list"].count()

227

In [6]:
count_dict = {}
for value in output:
    if value in count_dict:
        count_dict[value] += 1
    else:
        count_dict[value] = 1

# Print the counts
for key, value in count_dict.items():
    print(f"{key}: {value}")

I: 94
VI#: 5
v: 1
iii: 3
vi: 53
ii: 12
IV: 24
V: 11
I#: 1
III: 7
IV#: 1
II#: 6
II: 2
VIIb: 1
i: 1
V#: 2
vii: 3


In [7]:
newdata["Progression list"].value_counts()

Progression list
I V vi IV         13
II# v IV           6
vi I V II          6
I V ii IV          5
vi IV ii III       5
                  ..
I# vi# V#          1
I iii IV I V       1
I V IV vi V        1
I iii IV I V I     1
I IV               1
Name: count, Length: 133, dtype: int64

In [24]:
from torch.utils.data import Dataset, DataLoader
import torch

from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)


# Define the custom dataset and data loader
class ChordDataset(Dataset):
    def __init__(self, tokenized_dataset):
        self.tokenized_dataset = tokenized_dataset

    def __len__(self):
        return len(self.tokenized_dataset["input_ids"])

    def __getitem__(self, idx):
        return {
            "input_ids": self.tokenized_dataset["input_ids"][idx],
            "attention_mask": self.tokenized_dataset["attention_mask"][idx],
        }
    
training_data = list(newdata["Progression list"])

# Tokenize the dataset
tokenizer.pad_token = tokenizer.eos_token
tokenized_dataset = tokenizer(training_data, return_tensors="pt", padding=True, truncation=True)


# Fine-tune the GPT-2 model on the chord progression dataset
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Create an instance of the custom dataset and data loader
chord_dataset = ChordDataset(tokenized_dataset)
data_loader = DataLoader(chord_dataset, batch_size=2, shuffle=True)

In [25]:
num_epochs = 3
batch_size = 2

for epoch in range(num_epochs):
    for batch in data_loader:
        inputs = {
            "input_ids": batch["input_ids"].to(torch.device("cuda" if torch.cuda.is_available() else "cpu")),
            "attention_mask": batch["attention_mask"].to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
        }

        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

In [19]:
seed_chord = "I"
input_ids = tokenizer.encode(seed_chord, return_tensors="pt")
output = model.generate(
        input_ids,
        max_length=10,
        temperature=0.3,
        pad_token_id=tokenizer.eos_token_id,
        num_return_sequences=1,
    )

generated_chords = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_chords.split(' '))

c:\Users\mally\Leiden University [2022 -2024]\3rd Sem\API\Final Project\.venv\lib\site-packages\transformers\generation\configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


['I', 'V', 'vi', 'IV']


In [21]:
model.save_pretrained('fine_tuned_model_directory')
tokenizer.save_pretrained('fine_tuned_model_directory')

('fine_tuned_model_directory\\tokenizer_config.json',
 'fine_tuned_model_directory\\special_tokens_map.json',
 'fine_tuned_model_directory\\vocab.json',
 'fine_tuned_model_directory\\merges.txt',
 'fine_tuned_model_directory\\added_tokens.json')

In [20]:
from pydub import AudioSegment
from pydub.playback import play

def generate_guitar_chord(file_path, duration=1000):
    # Load guitar sample
    guitar_chord = AudioSegment.from_file(file_path)

    # Ensure the sample duration matches the desired duration
    guitar_chord = guitar_chord[:duration]

    return guitar_chord

# Chord progression: I, VI#, v, IV
chord_progression = generated_chords.split(' ')

# Path to guitar samples (replace with your actual file paths)
sample_paths = {
    'I': r'C:\Users\mally\Leiden University [2022 -2024]\3rd Sem\API\Final Project\C_AcousticGuitar_RodrigoMercador_1 (1).wav',
    'V': r'C:\Users\mally\Leiden University [2022 -2024]\3rd Sem\API\Final Project\G_AcousticGuitar_RodrigoMercador_1.wav',
    'vi': r'C:\Users\mally\Leiden University [2022 -2024]\3rd Sem\API\Final Project\Am_AcousticGuitar_RodrigoMercador_1.wav',
    'IV': r'C:\Users\mally\Leiden University [2022 -2024]\3rd Sem\API\Final Project\F_AcousticGuitar_RodrigoMercador_1.wav',
    # 'VI#':,
    'v':r"C:\Users\mally\Leiden University [2022 -2024]\3rd Sem\API\Final Project\guitar-chord-g-minor_G_minor.wav",
    'iii':r"C:\Users\mally\Leiden University [2022 -2024]\3rd Sem\API\Final Project\Em_AcousticGuitar_RodrigoMercador_1.wav",
    'ii':r"C:\Users\mally\Leiden University [2022 -2024]\3rd Sem\API\Final Project\Dm_AcousticGuitar_RodrigoMercador_1.wav",
    # 'I#':,
    # 'III':,
    # 'IV#':,
    # 'II#':,
    # 'II':,
    'VIIb':r"C:\Users\mally\Leiden University [2022 -2024]\3rd Sem\API\Final Project\Bdim_AcousticGuitar_RodrigoMercador_1.wav",
    # 'i':,
    # 'V#':,
    'vii':r"C:\Users\mally\Leiden University [2022 -2024]\3rd Sem\API\Final Project\Bb_AcousticGuitar_RodrigoMercador_1.wav"
}

# Generate audio for each chord and concatenate them
full_music = AudioSegment.silent(duration=0)
for chord_symbol in chord_progression:
    if chord_symbol in sample_paths:
        chord_audio = generate_guitar_chord(sample_paths[chord_symbol])
        full_music += chord_audio
    else:
        print(f"Chord '{chord_symbol}' not recognized.")

# Play the generated music
full_music.export("guitar_music_output.wav", format="wav")

<_io.BufferedRandom name='guitar_music_output.wav'>

In [3]:
import numpy as np
from midiutil import MIDIFile
import fluidsynth
import subprocess

def generate_chord_midi(filename, chord_frequencies, duration=4, tempo=120):
    midi = MIDIFile(1)
    midi.addTempo(0, 0, tempo)

    for i, freq in enumerate(chord_frequencies):
        midi.addNote(0, 0, i * 2, 60 + i, duration, 100)

    with open(filename, 'wb') as midi_file:
        midi.writeFile(midi_file)

def convert_midi_to_wav(midi_filename, wav_filename):
    fluidsynth_cmd = 'fluidsynth'
    soundfont_path = 'path/to/soundfont.sf2'  # replace with a path to a soundfont file on your system

    # Convert MIDI to WAV using FluidSynth
    subprocess.run([fluidsynth_cmd, '-ni', soundfont_path, midi_filename, '-F', wav_filename, '-r', '44100'])

if __name__ == '__main__':
    # A major chord with a raised sixth (VI#) in the key of C major
    chord_frequencies = [440.00, 554.37, 659.26]  # A, C#, E

    midi_filename = 'VIsharp_chord.mid'
    wav_filename = 'VIsharp_chord.wav'

    generate_chord_midi(midi_filename, chord_frequencies)
    convert_midi_to_wav(midi_filename, wav_filename)

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [2]:
!pip install fluidsynth

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for fluidsynth: filename=fluidsynth-0.2-py3-none-any.whl size=4496 sha256=76dba261677f8ed83c6cfeee92f8c92f8513c5f9a17cd9d33f05d9f611b92dd0
  Stored in directory: c:\users\mally\appdata\local\pip\cache\wheels\72\5a\bf\347b142f199fb1bea87f16f8970916226951bc359c947ba19b
Successfully built fluidsynth
